In [1]:
import numpy as np
import pandas as pd
import pandas_profiling
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
trade = pd.read_csv('./DSG2018-qualifiers/Trade.csv')

In [3]:
trade.sort_values(by=['IsinIdx','TradeDateKey'],inplace=True)

In [4]:
trade.head()

,TradeDateKey,CustomerIdx,IsinIdx,BuySell,NotionalEUR,Price,TradeStatus,CustomerInterest
308333,20160222,1537,0,Buy,1360668.0,64.0,Unknown,1.0
772478,20160222,1537,0,Buy,1360668.0,64.0,Unknown,1.0
326413,20160505,2926,0,Sell,779961.0,63.0,Unknown,1.0
1304482,20160520,2743,0,Sell,1533113.0,61.0,NotTraded,1.0
1304483,20160520,2743,0,Sell,1533113.0,61.0,NotTraded,1.0


In [5]:
trade['TradeDate'] = pd.to_datetime(trade.TradeDateKey.astype(str),format='%Y%m%d')

In [6]:
trade['week_of_year'] = trade.TradeDate.dt.weekofyear
trade['year'] = trade.TradeDate.dt.year
trade['prev_week'] = trade.week_of_year - 1
trade['prev_year'] = trade.year

In [7]:
trade.groupby('year')['week_of_year'].max()

year
2016    53
2017    52
2018    16
Name: week_of_year, dtype: int64

In [8]:
trade.loc[trade['prev_week'] == 0,'prev_year'] = trade.loc[trade['prev_week'] == 0,'year']-1
trade.loc[trade['prev_week'] == 0,'prev_week'] = trade.loc[trade['prev_week'] == 0,'prev_year'].transform(lambda x: (x%4==0)).astype('uint8') + 52

In [9]:
trade['weekYear_idx'] = trade.year * 100 + trade.week_of_year
trade['prev_weekYear_idx'] = trade.prev_year * 100 + trade.prev_week

In [10]:
trade.drop(['TradeDateKey','CustomerInterest','TradeDate','week_of_year','year','prev_week','prev_year'],axis=1,inplace=True)

In [11]:
trade.head()

,CustomerIdx,IsinIdx,BuySell,NotionalEUR,Price,TradeStatus,weekYear_idx,prev_weekYear_idx
308333,1537,0,Buy,1360668.0,64.0,Unknown,201608,201607
772478,1537,0,Buy,1360668.0,64.0,Unknown,201608,201607
326413,2926,0,Sell,779961.0,63.0,Unknown,201618,201617
1304482,2743,0,Sell,1533113.0,61.0,NotTraded,201620,201619
1304483,2743,0,Sell,1533113.0,61.0,NotTraded,201620,201619


In [12]:
trade = pd.get_dummies(trade,columns=['TradeStatus','BuySell'])

In [13]:
trade_grouped = trade.groupby(['IsinIdx','weekYear_idx','prev_weekYear_idx']).agg({
    'Price': ['mean','max','min'],
    'NotionalEUR': ['mean','std','min'],
    'TradeStatus_Done': 'mean',
    'TradeStatus_Holding': 'mean',
    'TradeStatus_IOI': 'mean',
    'TradeStatus_NotTraded': 'mean',
    'TradeStatus_Unknown': 'mean',
    'BuySell_Buy': ['mean','count']
})

In [14]:
trade_grouped = trade_grouped.reset_index()

In [15]:
trade_grouped.columns = [x+'_'+y if y!='' else x for x,y in zip(trade_grouped.columns.get_level_values(0),
               trade_grouped.columns.get_level_values(1))]

In [16]:
trade_grouped.head()

,IsinIdx,weekYear_idx,prev_weekYear_idx,Price_mean,Price_max,Price_min,NotionalEUR_mean,NotionalEUR_std,NotionalEUR_min,TradeStatus_Done_mean,TradeStatus_Holding_mean,TradeStatus_IOI_mean,TradeStatus_NotTraded_mean,TradeStatus_Unknown_mean,BuySell_Buy_mean,BuySell_Buy_count
0,0,201608,201607,64.000000,64.0,64.0,1.360668e+06,0.000000,1360668.0,0.000000,0.0,0.0,0.000000,1.000000,1.000000,2
1,0,201618,201617,63.000000,63.0,63.0,7.799610e+05,NaN,779961.0,0.000000,0.0,0.0,0.000000,1.000000,0.000000,1
2,0,201620,201619,61.000000,61.0,61.0,1.533113e+06,0.000000,1533113.0,0.000000,0.0,0.0,1.000000,0.000000,0.000000,2
3,0,201621,201620,62.851429,64.0,60.0,1.580173e+06,96745.077658,1533113.0,0.285714,0.0,0.0,0.428571,0.285714,0.285714,7
4,0,201626,201625,64.000000,64.0,64.0,9.088430e+05,0.000000,908843.0,0.000000,0.0,0.0,0.000000,1.000000,1.000000,2


In [17]:
trade_grouped.loc[:,
    ['Price_max','Price_min', 'Price_mean','NotionalEUR_std']
] = trade_grouped.loc[:,
    ['Price_max','Price_min', 'Price_mean','NotionalEUR_std']
].fillna(0)

In [18]:
trade_grouped.head()

,IsinIdx,weekYear_idx,prev_weekYear_idx,Price_mean,Price_max,Price_min,NotionalEUR_mean,NotionalEUR_std,NotionalEUR_min,TradeStatus_Done_mean,TradeStatus_Holding_mean,TradeStatus_IOI_mean,TradeStatus_NotTraded_mean,TradeStatus_Unknown_mean,BuySell_Buy_mean,BuySell_Buy_count
0,0,201608,201607,64.000000,64.0,64.0,1.360668e+06,0.000000,1360668.0,0.000000,0.0,0.0,0.000000,1.000000,1.000000,2
1,0,201618,201617,63.000000,63.0,63.0,7.799610e+05,0.000000,779961.0,0.000000,0.0,0.0,0.000000,1.000000,0.000000,1
2,0,201620,201619,61.000000,61.0,61.0,1.533113e+06,0.000000,1533113.0,0.000000,0.0,0.0,1.000000,0.000000,0.000000,2
3,0,201621,201620,62.851429,64.0,60.0,1.580173e+06,96745.077658,1533113.0,0.285714,0.0,0.0,0.428571,0.285714,0.285714,7
4,0,201626,201625,64.000000,64.0,64.0,9.088430e+05,0.000000,908843.0,0.000000,0.0,0.0,0.000000,1.000000,1.000000,2


,CustomerIdx,IsinIdx,NotionalEUR,Price,weekYear_idx,prev_weekYear_idx,TradeStatus_Done,TradeStatus_Holding,TradeStatus_IOI,TradeStatus_NotTraded,TradeStatus_Unknown,BuySell_Buy,BuySell_Sell
308333,1537,0,1360668.0,64.0,201608,201607,0,0,0,0,1,1,0
772478,1537,0,1360668.0,64.0,201608,201607,0,0,0,0,1,1,0
326413,2926,0,779961.0,63.0,201618,201617,0,0,0,0,1,0,1
1304482,2743,0,1533113.0,61.0,201620,201619,0,0,0,1,0,0,1
1304483,2743,0,1533113.0,61.0,201620,201619,0,0,0,1,0,0,1
